In [4]:
# Libraries for feature extraction:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from tqdm import tqdm


In [5]:
#Audio data directory:
audio_dir = './Data/genres/'

In [6]:
#Dictionary for genres label encoding:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [7]:
#Get selected features from each song using librosa and numpy:

def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Selected features:
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None, 'chroma':None}
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # Treatment of MFCC feature
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Calculate statistics for each feature:
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
        return result
    
    dict_agg_features = get_moments(features)
    
    #Calculating one more feature:
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [8]:
#Reading all audio files and calculating features for each of them. This function generates a row for each song in the directory:

def read_process_songs(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)

                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))

                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = genres[x]
                arr_features.append(features)
    return arr_features

In [9]:
#Reading all audio files and calculating features for each of them. 
#This function generates a 2 rows for each song, separating each of them into 2 15s audio clips:

def read_process_songs2(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal1, sr1 = librosa.load(file_name,duration=15)
                signal2, sr2 = librosa.load(file_name,offset=15)
                samples=[(signal1,sr1),(signal2,sr2)]

                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))

                # Append the result to the data structure
                for s in samples:
                    features = get_features(s[0], s[1])
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [10]:
#Reading all audio files and calculating features for each of them. 
#This function generates a 2 rows for each song, separating each of them into 2 15s audio clips:

def read_process_songs3(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
                samples=[(signal[:220000],sr),(signal[220000:440000],sr),(signal[440000:],sr)]

                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))

                # Append the result to the data structure
                for s in samples:
                    features = get_features(s[0], s[1])
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [25]:
# Get list of dicts with features and convert to dataframe (1 row per song)
features = read_process_songs(audio_dir,debug=False)

In [26]:
# Get list of dicts with features and convert to dataframe (2 rows per song, each song split in half):
features2 = read_process_songs2(audio_dir,debug=False)

In [10]:
#Convert to dataframe:
df_features1 = pd.DataFrame(features)

In [11]:
df_features1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 40 columns):
centroid_mean    990 non-null float64
centroid_std     990 non-null float64
chroma_mean      990 non-null float64
chroma_std       990 non-null float64
flux_mean        990 non-null float64
flux_std         990 non-null float64
genre            990 non-null int64
mfcc_0_mean      990 non-null float64
mfcc_0_std       990 non-null float64
mfcc_10_mean     990 non-null float64
mfcc_10_std      990 non-null float64
mfcc_11_mean     990 non-null float64
mfcc_11_std      990 non-null float64
mfcc_12_mean     990 non-null float64
mfcc_12_std      990 non-null float64
mfcc_1_mean      990 non-null float64
mfcc_1_std       990 non-null float64
mfcc_2_mean      990 non-null float64
mfcc_2_std       990 non-null float64
mfcc_3_mean      990 non-null float64
mfcc_3_std       990 non-null float64
mfcc_4_mean      990 non-null float64
mfcc_4_std       990 non-null float64
mfcc_5_mean      990 no

In [19]:
df_features1.describe()

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,genre,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
count,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,...,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000
mean,2101.633293,651.510202,0.453753,0.290499,1.417293,1.554270,4.500000,-216.532282,59.482719,-5.555799,...,10.043930,7.865160,9.707282,0.128891,0.053661,4360.189433,1325.849435,121.042726,0.103695,0.054597
std,700.008768,280.471797,0.088156,0.018794,0.340532,0.678716,2.873733,97.954624,21.046591,6.788720,...,2.003818,7.874301,1.923625,0.064573,0.029559,1557.596573,523.244123,20.918251,0.041944,0.026151
min,534.046982,73.853662,0.211247,0.209446,0.608654,0.163264,0.000000,-617.812439,13.544660,-26.293921,...,6.103590,-12.293718,5.337916,0.005265,0.004148,730.280348,124.919782,69.837416,0.021691,0.007039
25%,1525.887998,438.330101,0.392470,0.281165,1.161509,1.075469,2.000000,-272.108604,44.643503,-10.491795,...,8.490852,1.835600,8.304592,0.085169,0.034457,3133.797163,917.915046,103.359375,0.070116,0.035725
50%,2112.958377,600.494099,0.460690,0.292650,1.363733,1.448091,4.500000,-192.064430,58.017269,-5.530820,...,9.807375,7.411592,9.443274,0.119777,0.048122,4483.452704,1272.714350,117.453835,0.099473,0.049433
75%,2623.693136,814.389822,0.511269,0.302494,1.630967,1.981211,7.000000,-147.179733,72.448988,-0.592356,...,11.306490,14.226377,10.839503,0.173146,0.065561,5349.056660,1672.126680,135.999178,0.132425,0.066568
max,4171.532799,1921.069855,0.689486,0.342938,2.450388,4.371769,9.000000,-47.043102,166.846832,17.014378,...,18.426645,27.413534,19.880682,0.394304,0.176083,8569.415467,3423.674142,184.570312,0.274505,0.186829


In [20]:
df_features1.groupby('genre').mean()

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,mfcc_10_std,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
genre,,,,,,,,,,,,,,,,,,,,,
0,1649.174129,521.729407,0.429335,0.300208,1.391695,1.526578,-235.647698,58.470357,-9.232909,9.524527,...,10.207779,8.241184,9.855519,0.134283,0.057289,3458.259303,1176.362606,124.198627,0.079003,0.042369
1,1294.469166,325.540346,0.318281,0.296181,0.999830,0.726121,-385.371123,69.986975,-6.080446,10.641368,...,10.076955,0.999549,10.111902,0.042918,0.023600,2348.734258,712.395030,125.937531,0.077573,0.028184
2,1780.211424,591.585561,0.432005,0.291483,1.256552,1.327519,-214.801876,54.211695,-6.355206,9.487279,...,10.545299,6.568730,10.322946,0.123982,0.043008,3700.235699,1318.532011,120.419897,0.083142,0.042769
3,2510.464256,729.988728,0.476346,0.286887,1.585123,1.873694,-168.231954,57.700228,-6.598454,8.997502,...,9.518864,10.765014,9.269775,0.133183,0.058948,5294.735666,1424.064066,120.014979,0.129522,0.064511
4,2427.412052,886.843760,0.528858,0.288674,1.818005,2.064275,-180.829765,67.659508,-1.928479,9.916088,...,11.032898,10.395469,10.381075,0.174293,0.087020,5133.454403,1623.483811,110.234545,0.108029,0.076417
5,1707.152407,489.037820,0.374921,0.305565,1.210625,1.265180,-282.082079,56.132658,-3.819220,8.707425,...,8.971317,3.913932,8.618519,0.086166,0.037426,3540.388320,1161.495485,119.535055,0.077274,0.037811
6,2538.286261,465.981696,0.548349,0.262834,1.170334,0.993727,-135.323540,36.610158,-10.858198,7.564052,...,8.330931,16.851456,7.970945,0.151905,0.038245,5035.929491,870.725973,123.648112,0.147848,0.047562
7,2926.187181,997.189077,0.459242,0.297994,1.627127,1.948215,-142.400219,65.470358,-2.097397,9.893075,...,10.885800,4.274340,10.558405,0.196715,0.089142,6351.014439,1847.180973,113.377624,0.133928,0.087894
8,2035.329856,878.192527,0.505532,0.290142,1.829774,2.431211,-240.482293,76.250681,-0.568495,10.460885,...,11.470647,7.804395,11.109662,0.118692,0.059269,4262.257737,1806.079774,132.898062,0.091066,0.066675


In [12]:
#Save as .csv:
df_features1.to_csv('./Features/features1.csv', index=False)


In [16]:
df_features2=pd.DataFrame(features2)

In [17]:
df_features2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 40 columns):
centroid_mean    1980 non-null float64
centroid_std     1980 non-null float64
chroma_mean      1980 non-null float64
chroma_std       1980 non-null float64
flux_mean        1980 non-null float64
flux_std         1980 non-null float64
genre            1980 non-null int64
mfcc_0_mean      1980 non-null float64
mfcc_0_std       1980 non-null float64
mfcc_10_mean     1980 non-null float64
mfcc_10_std      1980 non-null float64
mfcc_11_mean     1980 non-null float64
mfcc_11_std      1980 non-null float64
mfcc_12_mean     1980 non-null float64
mfcc_12_std      1980 non-null float64
mfcc_1_mean      1980 non-null float64
mfcc_1_std       1980 non-null float64
mfcc_2_mean      1980 non-null float64
mfcc_2_std       1980 non-null float64
mfcc_3_mean      1980 non-null float64
mfcc_3_std       1980 non-null float64
mfcc_4_mean      1980 non-null float64
mfcc_4_std       1980 non-null float6

In [21]:
df_features2.describe()

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,genre,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
count,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,...,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000
mean,2101.558240,639.050011,0.453808,0.289594,1.416059,1.547285,4.500000,-216.869493,56.843673,-5.688760,...,9.859573,8.013199,9.549670,0.128882,0.052513,4360.002880,1301.778379,121.276821,0.103653,0.053397
std,708.601100,286.974973,0.090838,0.019949,0.344241,0.694053,2.873007,100.165628,20.993436,7.044022,...,2.089142,8.063983,2.037660,0.065427,0.029687,1574.057491,533.158762,21.157112,0.042932,0.026970
min,528.695515,43.333355,0.126825,0.208786,0.559117,0.159940,0.000000,-653.586731,9.547230,-27.604580,...,5.667604,-17.415010,5.133595,0.003434,0.001955,716.553112,117.940624,63.024009,0.021119,0.006806
25%,1530.083938,423.411856,0.393762,0.278927,1.158767,1.060243,2.000000,-273.499741,42.148064,-10.781512,...,8.281676,1.799230,8.084083,0.085034,0.032853,3121.382684,904.837254,103.359375,0.069603,0.034104
50%,2116.458414,591.284539,0.461585,0.292046,1.356086,1.434154,4.500000,-193.411926,55.430264,-5.548859,...,9.626751,7.718230,9.276059,0.119796,0.047165,4464.828128,1238.516183,117.453835,0.099513,0.047840
75%,2614.461215,809.720140,0.513838,0.302552,1.624592,1.970705,7.000000,-145.270535,69.967079,-0.485506,...,11.108516,14.815290,10.706733,0.172842,0.065846,5377.469660,1647.756139,135.999178,0.133689,0.066579
max,4600.185306,1974.885127,0.703675,0.346537,2.540177,4.458624,9.000000,-20.707417,179.063538,17.388300,...,19.415730,35.408268,22.664005,0.419707,0.175881,9219.539099,3401.919678,184.570312,0.322515,0.188076


In [22]:
df_features2.groupby('genre').mean()

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,mfcc_10_std,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
genre,,,,,,,,,,,,,,,,,,,,,
0,1649.166358,513.871317,0.429333,0.299494,1.391943,1.524789,-236.201765,57.450334,-9.496317,9.420878,...,10.079044,8.544552,9.730911,0.134280,0.056501,3458.183768,1159.761565,122.972046,0.078974,0.041548
1,1294.474279,302.267950,0.318706,0.295385,1.009291,0.717400,-387.010268,62.405932,-6.403488,10.229843,...,9.629743,1.377254,9.772892,0.042916,0.021419,2348.746084,667.593928,125.408352,0.077541,0.026442
2,1780.145658,582.002577,0.432255,0.290968,1.254515,1.323211,-215.052598,52.099552,-6.476244,9.349416,...,10.392176,6.701897,10.183280,0.123980,0.041942,3700.080679,1299.261805,121.709202,0.083107,0.041746
3,2510.376417,722.083473,0.476409,0.286070,1.581576,1.863743,-168.317491,56.181447,-6.674549,8.904292,...,9.401271,10.847339,9.149089,0.133179,0.058569,5294.632759,1411.858852,120.485812,0.129465,0.063502
4,2427.159708,875.882914,0.528740,0.287895,1.814285,2.056589,-180.985945,65.727346,-1.975569,9.804362,...,10.886732,10.445985,10.255496,0.174228,0.086020,5132.764345,1601.513892,110.133916,0.107997,0.075283
5,1707.061399,474.332042,0.374926,0.304241,1.209307,1.260488,-282.394911,53.575919,-4.008376,8.475389,...,8.696112,4.122431,8.380917,0.086168,0.036737,3540.136717,1130.588309,120.483552,0.077241,0.036403
6,2538.251652,454.675197,0.548536,0.261423,1.168228,0.988953,-135.430344,34.826055,-10.977199,7.434000,...,8.219835,16.974509,7.845548,0.151905,0.037282,5035.874302,851.545341,125.335122,0.147795,0.046482
7,2926.063427,979.359581,0.459444,0.297095,1.623445,1.931345,-142.430812,61.668673,-2.100035,9.741295,...,10.705618,4.274415,10.412562,0.196703,0.086773,6350.741829,1810.167016,113.353968,0.133868,0.086146
8,2035.275286,870.916590,0.505460,0.289376,1.827014,2.426273,-240.595669,75.325189,-0.629795,10.377813,...,11.370485,7.871084,11.040129,0.118694,0.058832,4262.070521,1793.890193,132.807573,0.091025,0.065916


In [23]:
df_features2.to_csv('./Features/features2.csv', index=False)

In [7]:
test_dir='./Data/test_songs/'

In [8]:
features_test = read_process_songs2(test_dir,debug=True)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav


In [9]:
df_test=pd.DataFrame(features_test)

In [10]:
df_test

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,genre,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
0,1348.963434,433.667640,0.452791,0.292625,1.096600,1.278877,0,-265.709045,84.141609,-7.775651,...,8.653544,4.979832,9.925613,0.093251,0.064832,2789.183134,1001.723758,92.285156,0.062706,0.038032
1,1608.404816,489.450162,0.439207,0.299258,1.140624,1.173483,0,-231.575439,108.281807,-7.034888,...,9.398550,7.466754,8.743001,0.114787,0.081587,3223.856652,1106.091422,92.285156,0.086654,0.043395
2,1488.843478,255.614769,0.386038,0.271425,0.884328,0.555000,1,-447.607300,21.722713,1.453901,...,7.110258,26.344177,7.834567,0.010196,0.001885,3224.580501,661.526740,129.199219,0.080539,0.023962
3,1457.377723,222.853448,0.393994,0.270746,0.866227,0.538925,1,-438.144379,22.568977,6.507122,...,7.713020,15.361012,7.232686,0.010761,0.002437,3116.889550,632.077510,117.453835,0.080736,0.020215
4,1974.335783,592.945482,0.467754,0.279877,1.398469,1.610231,2,-122.529060,43.560539,-12.834041,...,10.115583,15.441654,8.186692,0.208015,0.074243,4213.607854,1148.757941,143.554688,0.095036,0.055642
5,2165.633840,583.283737,0.444168,0.285833,1.302027,1.264878,2,-103.437935,47.385139,-15.827991,...,10.343629,17.751516,10.024124,0.227325,0.074285,4424.657222,1178.263843,143.554688,0.121290,0.057808
6,2283.209030,757.631086,0.534701,0.265781,1.897929,2.206208,3,-171.118729,50.930534,-12.501665,...,9.379219,17.207703,9.233993,0.119810,0.048018,5065.736035,1518.892110,123.046875,0.113626,0.068830
7,2267.173125,593.146282,0.465593,0.285898,1.828545,2.100000,3,-168.275375,47.700920,-11.521920,...,9.226274,17.610487,8.665509,0.119973,0.046421,5035.008709,1317.297781,123.046875,0.111364,0.047579
8,1814.941683,816.407124,0.473115,0.311546,1.778642,2.093600,4,-208.181290,63.086060,-5.794276,...,17.076347,9.942732,13.004704,0.148962,0.123663,3484.378930,1523.970700,107.666016,0.097260,0.069366
9,1997.022081,883.066444,0.469989,0.314936,1.982870,2.316374,4,-229.269470,65.380653,-4.012559,...,13.175270,6.996458,13.460243,0.145886,0.135194,3857.272296,1561.685989,107.666016,0.105663,0.078716


In [11]:
df_test.to_csv('./Features/features_test_2.csv', index=False)

In [12]:
features_test = read_process_songs(test_dir,debug=True)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav


In [13]:
df_test=pd.DataFrame(features_test)

In [14]:
df_test

,centroid_mean,centroid_std,chroma_mean,chroma_std,flux_mean,flux_std,genre,mfcc_0_mean,mfcc_0_std,mfcc_10_mean,...,mfcc_8_std,mfcc_9_mean,mfcc_9_std,rmse_mean,rmse_std,roloff_mean,roloff_std,tempo,zcr_mean,zcr_std
0,1478.754833,480.271532,0.444999,0.295968,1.123352,1.228280,0,-248.410645,98.362694,-7.201980,...,9.159321,5.998701,9.489829,0.104034,0.074492,3006.504790,1077.207601,92.285156,0.074744,0.042599
1,1472.965682,240.199445,0.390024,0.271111,0.876369,0.544919,1,-442.614166,22.561705,4.280841,...,9.844658,20.536285,9.144958,0.010479,0.002197,3170.343655,649.128638,117.453835,0.080679,0.022168
2,2070.027131,595.866892,0.455872,0.283083,1.352201,1.447242,2,-112.926453,46.478870,-14.274925,...,10.276713,16.525242,9.236507,0.217664,0.074883,4319.164189,1168.261734,143.554688,0.108231,0.058231
3,2275.306654,680.283091,0.499668,0.278059,1.863320,2.152533,3,-169.657928,49.342915,-11.983969,...,9.417696,17.386265,8.947528,0.119896,0.047219,5050.560334,1421.564340,123.046875,0.112548,0.059134
4,1906.033063,855.284366,0.470630,0.312785,1.892172,2.230346,4,-218.719254,65.105011,-4.890142,...,15.245326,8.443139,13.323622,0.147418,0.129574,3671.053079,1554.492515,107.666016,0.101494,0.074307
5,1398.301721,517.121162,0.401876,0.316449,1.231026,1.362276,5,-384.136444,45.092361,-4.148621,...,7.890797,10.737982,8.660866,0.038501,0.021962,3087.041967,1523.948388,135.999178,0.046178,0.025973
6,2295.913006,590.275388,0.620041,0.255773,1.319884,1.701510,6,-140.212128,50.396683,-6.873109,...,8.848748,12.419588,9.088333,0.166217,0.071480,4786.349763,1042.387186,143.554688,0.107930,0.061728
7,2562.646755,934.621812,0.439889,0.300191,1.502260,2.111934,7,-192.079193,68.561852,-7.996088,...,10.628657,9.033154,10.971423,0.136540,0.054561,6379.656912,2240.864820,161.499023,0.095967,0.065598
8,2302.185216,694.196228,0.486601,0.277635,1.632342,1.838657,8,-140.699875,54.753479,-1.392343,...,9.788798,8.270998,7.890015,0.178866,0.044140,4505.152443,870.169371,117.453835,0.139593,0.078594
9,1505.409169,645.034600,0.462147,0.269038,1.016052,1.035057,9,-299.567047,85.976105,-1.837569,...,10.086555,17.495628,9.728044,0.054303,0.018768,3035.731991,1353.471782,123.046875,0.076306,0.047798


In [15]:
df_test.to_csv('./Features/features_test_1.csv', index=False)

In [11]:
features3 = read_process_songs3(audio_dir,debug=True)

Reading file: ./Data/genres/blues/blues.00088.wav
Reading file: ./Data/genres/blues/blues.00091.wav
Reading file: ./Data/genres/blues/blues.00023.wav
Reading file: ./Data/genres/blues/blues.00024.wav
Reading file: ./Data/genres/blues/blues.00070.wav
Reading file: ./Data/genres/blues/blues.00053.wav
Reading file: ./Data/genres/blues/blues.00085.wav
Reading file: ./Data/genres/blues/blues.00033.wav
Reading file: ./Data/genres/blues/blues.00019.wav
Reading file: ./Data/genres/blues/blues.00031.wav
Reading file: ./Data/genres/blues/blues.00020.wav
Reading file: ./Data/genres/blues/blues.00001.wav
Reading file: ./Data/genres/blues/blues.00046.wav
Reading file: ./Data/genres/blues/blues.00098.wav
Reading file: ./Data/genres/blues/blues.00017.wav
Reading file: ./Data/genres/blues/blues.00009.wav
Reading file: ./Data/genres/blues/blues.00040.wav
Reading file: ./Data/genres/blues/blues.00074.wav
Reading file: ./Data/genres/blues/blues.00086.wav
Reading file: ./Data/genres/blues/blues.00068.wav


In [ ]:
df_test.to_csv('./Features/features_test_1.csv', index=False)

In [12]:
df3=pd.DataFrame(features3)

In [14]:
df3.to_csv('./Features/features3.csv', index=False)